In [1]:
%autoreload 2

In [2]:
import logging 
logging.basicConfig()

# logging.getLogger("sql").setLevel("DEBUG")
# logging.getLogger("gammas").setLevel("DEBUG")
# logging.getLogger("expectation_step").setLevel("DEBUG")
# logging.getLogger("maximisation_step").setLevel("DEBUG")
# logging.getLogger("comparison_evaluation").setLevel("DEBUG")
# logging.getLogger("sparklink").setLevel(logging.DEBUG)

In [3]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import StructType
import pyspark.sql.functions as f

# WARNING:
# These config options are appropriate only if you're running Spark locally!!!
conf=SparkConf()
conf.set('spark.driver.memory', '8g')
conf.set("spark.sql.shuffle.partitions", "8") 

sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession(sc)

In [4]:
from sparklink.sql import *
from sparklink.blocking import *
from sparklink.gammas import *
from sparklink.params import *
from sparklink.expectation_step import *
from sparklink.maximisation_step import *
from sparklink.iterate import *
from sparklink.comparison_evaluation import *

In [5]:
df = spark.read.csv("data/data_null.csv", header=True)
df.show()

+---------+---+-------+-----+
|unique_id|mob|surname|group|
+---------+---+-------+-----+
|        1| 10|Linacre|    1|
|        2| 10|Linacre|    1|
|        3| 10|Linacer|    1|
|        4|  7|  Smith|    2|
|        5|  8|  Smith|    2|
|        6|  8|  Smith|    2|
|        7|  8|   null|    2|
|        8|  8|  Jones|    3|
+---------+---+-------+-----+



In [6]:
# df_comparison = cartestian_block(df, spark=spark)

blocking_rules = []

blocking_rules.append('l.surname = r.surname')
blocking_rules.append('l.mob = r.mob')


df_comparison = block_using_rules(df, blocking_rules, spark=spark)

df_comparison = df_comparison.withColumn("label", (df_comparison["group_l"]==df_comparison["group_r"]).cast("int"))

df_comparison.show()

+-----------+-----------+-----+-----+---------+---------+-------+-------+-----+
|unique_id_l|unique_id_r|mob_l|mob_r|surname_l|surname_r|group_l|group_r|label|
+-----------+-----------+-----+-----+---------+---------+-------+-------+-----+
|          4|          5|    7|    8|    Smith|    Smith|      2|      2|    1|
|          5|          6|    8|    8|    Smith|    Smith|      2|      2|    1|
|          7|          8|    8|    8|     null|    Jones|      2|      3|    0|
|          1|          2|   10|   10|  Linacre|  Linacre|      1|      1|    1|
|          1|          3|   10|   10|  Linacre|  Linacer|      1|      1|    1|
|          5|          8|    8|    8|    Smith|    Jones|      2|      3|    0|
|          5|          7|    8|    8|    Smith|     null|      2|      2|    1|
|          4|          6|    7|    8|    Smith|    Smith|      2|      2|    1|
|          2|          3|   10|   10|  Linacre|  Linacer|      1|      1|    1|
|          6|          8|    8|    8|   

In [7]:
gamma_settings = {
    "mob": {
        "levels": 2
    },
    "surname": {
        "levels": 3
    }}

df_gammas = add_gammas(df_comparison, gamma_settings, spark, include_orig_cols = True)
df_gammas.show()

+-----------+-----------+-----+-----+---------+---------+-------+-------+
|unique_id_l|unique_id_r|mob_l|mob_r|surname_l|surname_r|gamma_0|gamma_1|
+-----------+-----------+-----+-----+---------+---------+-------+-------+
|          4|          5|    7|    8|    Smith|    Smith|      0|      2|
|          5|          6|    8|    8|    Smith|    Smith|      1|      2|
|          7|          8|    8|    8|     null|    Jones|      1|     -1|
|          1|          2|   10|   10|  Linacre|  Linacre|      1|      2|
|          1|          3|   10|   10|  Linacre|  Linacer|      1|      1|
|          5|          8|    8|    8|    Smith|    Jones|      1|      0|
|          5|          7|    8|    8|    Smith|     null|      1|     -1|
|          4|          6|    7|    8|    Smith|    Smith|      0|      2|
|          2|          3|   10|   10|  Linacre|  Linacer|      1|      1|
|          6|          8|    8|    8|    Smith|    Jones|      1|      0|
|          6|          7|    8|    8| 

In [8]:
import copy 

params = Params(gamma_settings, starting_lambda=0.2)



In [9]:
import json
# print(json.dumps(params.params, indent=4, ensure_ascii=False))

In [10]:
sql = sql_gen_gamma_prob_columns(params)

df_gammas.registerTempTable("df_with_gamma")
df_with_gamma_probs = spark.sql(sql)
df_with_gamma_probs.show()

+-----------+-----------+-----+-----+---------+---------+-------+-------+----------------------+------------------+----------------------+------------------+
|unique_id_l|unique_id_r|mob_l|mob_r|surname_l|surname_r|gamma_0|gamma_1|prob_gamma_0_non_match|prob_gamma_0_match|prob_gamma_1_non_match|prob_gamma_1_match|
+-----------+-----------+-----+-----+---------+---------+-------+-------+----------------------+------------------+----------------------+------------------+
|          4|          5|    7|    8|    Smith|    Smith|      0|      2|                   0.9|               0.1|                   0.1|               0.7|
|          5|          6|    8|    8|    Smith|    Smith|      1|      2|                   0.1|               0.9|                   0.1|               0.7|
|          7|          8|    8|    8|     null|    Jones|      1|     -1|                   0.1|               0.9|                   1.0|               1.0|
|          1|          2|   10|   10|  Linacre|  Lin

In [11]:
df_with_gamma_probs.registerTempTable("df_with_gamma_probs")
sql = sql_gen_expected_match_prob(params)

df_e = spark.sql(sql)
df_e.show()

+-----------+-----------+-----+-----+---------+---------+-------+-------+----------------------+------------------+----------------------+------------------+-----------------+
|unique_id_l|unique_id_r|mob_l|mob_r|surname_l|surname_r|gamma_0|gamma_1|prob_gamma_0_non_match|prob_gamma_0_match|prob_gamma_1_non_match|prob_gamma_1_match|match_probability|
+-----------+-----------+-----+-----+---------+---------+-------+-------+----------------------+------------------+----------------------+------------------+-----------------+
|          4|          5|    7|    8|    Smith|    Smith|      0|      2|                   0.9|               0.1|                   0.1|               0.7|  0.1627906976744|
|          5|          6|    8|    8|    Smith|    Smith|      1|      2|                   0.1|               0.9|                   0.1|               0.7|  0.9402985074627|
|          7|          8|    8|    8|     null|    Jones|      1|     -1|                   0.1|               0.9|     

In [31]:
# It's saying that for each row where surname_l == surname_r, compute the proportion of matches and non-matches

# Note: For an uncommon surname, we expect most to be matches (i.e. the fields other than surname will mostly match)
# For a common surname, we expect many to be non-matches (i.e. the fields other than surname will often not match)
# This is the mechnamism by which term frequency gives us more accurate results for uncommon tokens.

# Need to repeat this step for each column where we're performing the adjustment

df_e.registerTempTable("df_e")

sql = """
select surname_l, surname_r, sum(match_probability) mp, sum(1-match_probability) as nmp
from df_e
where surname_l = surname_r
group by surname_l, surname_r

"""
surname_lookup = spark.sql(sql)

surname_lookup.registerTempTable("surname_lookup")


In [34]:

sql = f"""
select *, coalesce(mp/(mp+nmp), {params.params["λ"]}) as pseudo_lambda
from df_e as e 
left join
surname_lookup as s
on s.surname_l = e.surname_l
and s.surname_l = e.surname_r
"""
df_e_adj = spark.sql(sql)
df_e_adj.registerTempTable("df_e_adj")

sql = """
select *, (pseudo_lambda * prob_gamma_0_match * prob_gamma_1_match) /((pseudo_lambda * prob_gamma_0_match * prob_gamma_1_match) + ((1-pseudo_lambda) * prob_gamma_0_non_match * prob_gamma_1_non_match)) as adjusted_match_prob
from 
df_e_adj
"""

spark.sql(sql).show()

+-----------+-----------+-----+-----+---------+---------+-------+-------+----------------------+------------------+----------------------+------------------+-----------------+---------+---------+---------------+--------------+-------------+-------------------+
|unique_id_l|unique_id_r|mob_l|mob_r|surname_l|surname_r|gamma_0|gamma_1|prob_gamma_0_non_match|prob_gamma_0_match|prob_gamma_1_non_match|prob_gamma_1_match|match_probability|surname_l|surname_r|             mp|           nmp|pseudo_lambda|adjusted_match_prob|
+-----------+-----------+-----+-----+---------+---------+-------+-------+----------------------+------------------+----------------------+------------------+-----------------+---------+---------+---------------+--------------+-------------+-------------------+
|          4|          5|    7|    8|    Smith|    Smith|      0|      2|                   0.9|               0.1|                   0.1|               0.7|  0.1627906976744|    Smith|    Smith|1.2658799028115|1.7341